# Urban 8k Audio Classification Project

## Dataset Description
This dataset contains 8732 labeled sound excerpts (<=4s) of urban sounds from 10 classes: air_conditioner, car_horn, children_playing, dog_bark, drilling, enginge_idling, gun_shot, jackhammer, siren, and street_music. The classes are drawn from the urban sound taxonomy. For a detailed description of the dataset and how it was compiled please refer to our paper.
All excerpts are taken from field recordings uploaded to www.freesound.org. The files are pre-sorted into ten folds (folders named fold1-fold10) to help in the reproduction of and comparison with the automatic classification results reported in the article above.

In addition to the sound excerpts, a CSV file containing metadata about each excerpt is also provided.

**8732 audio files of urban sounds (see description above) in WAV format. The sampling rate, bit depth, and number of channels are the same as those of the original file uploaded to Freesound (and hence may vary from file to file).**

## Task Description
The Dataset contains 10 classes, we will do classification task on the audio file obtained.
```
A numeric identifier of the sound class:
0 = air_conditioner
1 = car_horn
2 = children_playing
3 = dog_bark
4 = drilling
5 = engine_idling
6 = gun_shot
7 = jackhammer
8 = siren
9 = street_music
```

In [ ]:
!pip install opendatasets --upgrade --quiet

In [ ]:
import opendatasets as od

dataset_url = "https://www.kaggle.com/chrisfilo/urbansound8k"
od.download(dataset_url)

100%|██████████| 5.61G/5.61G [00:48<00:00, 123MB/s]


In [ ]:
import librosa
import pandas as pd
import os
import glob
import numpy as np
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout

import IPython.display as ipd

In [ ]:
csv_file_path = glob.glob(os.path.join("./urbansound8k/*.csv"), recursive=True)

df = pd.read_csv(csv_file_path[0])
df.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [ ]:
print(f"Shape of the dataset: {df.shape}")

Shape of the dataset: (8732, 8)


In [ ]:
# The code extracts Mel-frequency cepstral coefficients (MFCC) features from audio files
# located in the specified path, and returns the features and their corresponding labels.

import os
import numpy as np
import librosa
from tqdm.notebook import tqdm

# Set the path where the audio files are located
path = csv_file_path[0]


def features_extract(file):
    """
    This function loads an audio file, extracts MFCC features,
    scales the features, and returns the scaled features.

    Args:
        file (str): The path of the audio file.

    Returns:
        numpy.ndarray: The scaled MFCC features.
    """
    # Load the audio file
    audio, sample_rate = librosa.load(file)

    # Extract MFCC features
    feature = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=50)

    # Scale the features
    scaled_feature = np.mean(feature.T, axis=0)

    return scaled_feature


# Create an empty list to store the extracted features and their labels
extracted = []

# Loop through the DataFrame and extract features and labels
for index_num, row in tqdm(df.iterrows()):
    # Create the path to the audio file
    file_name = os.path.join(
        "/content/urbansound8k/",
        "fold" + str(row["fold"]) + "/",
        str(row["slice_file_name"]),
    )

    # Get the label of the audio file
    final_class_labels = row["class"]

    # Extract the MFCC features and append to the extracted list
    data = features_extract(file_name)
    extracted.append([data, final_class_labels])

0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=1323
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=1523
  warnings.warn(


In [ ]:
len(extracted)

8732

In [ ]:
print(f"Extracted 'Feature' from dataset:\n{extracted[0][0]}")
print(f"\nExtracted class from dataset: {extracted[0][1]}")

Extracted 'Feature' from dataset:
[-2.11936981e+02  6.25812149e+01 -1.22813156e+02 -6.07452927e+01
 -1.38937550e+01 -2.97898350e+01 -3.97887087e+00  1.16827412e+01
  1.29638319e+01  8.33642006e+00 -6.52981663e+00 -2.99610305e+00
 -1.28879461e+01  5.88593531e+00 -2.47522211e+00 -4.40830946e+00
 -1.50144749e+01 -9.43993688e-01 -2.57990861e+00  6.09830046e+00
 -1.07862263e+01 -8.06347752e+00  5.84409714e+00 -3.12253737e+00
 -5.98421907e+00 -7.99421892e-02 -3.68708348e+00  4.74768400e+00
 -6.53008938e+00 -5.08387709e+00  4.87733096e-01  7.51088953e+00
 -8.85352790e-01  2.39381790e+00 -6.39236736e+00 -2.13185430e+00
  2.27601957e+00 -7.91079581e-01 -1.54006314e+00  1.32115340e+00
  5.34525514e-01  1.85582459e+00 -4.75857115e+00 -4.39922333e+00
 -5.72198153e+00  3.75011295e-01 -5.80002642e+00  1.23093772e+00
 -5.30414248e+00  2.22271156e+00]

Extracted class from dataset: dog_bark


In [ ]:
ext_df = pd.DataFrame(data=extracted, columns=["feature", "class"])
ext_df.head()

,feature,class
0,"[-211.93698, 62.581215, -122.813156, -60.74529...",dog_bark
1,"[-417.0052, 99.336624, -42.995586, 51.073326, ...",children_playing
2,"[-452.39316, 112.36253, -37.578075, 43.195866,...",children_playing
3,"[-406.47922, 91.1966, -25.043556, 42.78452, 11...",children_playing
4,"[-439.63873, 103.86223, -42.658787, 50.690285,...",children_playing


In [ ]:
ext_df["feature"].tolist()

[array([-2.11936981e+02,  6.25812149e+01, -1.22813156e+02, -6.07452927e+01,
        -1.38937550e+01, -2.97898350e+01, -3.97887087e+00,  1.16827412e+01,
         1.29638319e+01,  8.33642006e+00, -6.52981663e+00, -2.99610305e+00,
        -1.28879461e+01,  5.88593531e+00, -2.47522211e+00, -4.40830946e+00,
        -1.50144749e+01, -9.43993688e-01, -2.57990861e+00,  6.09830046e+00,
        -1.07862263e+01, -8.06347752e+00,  5.84409714e+00, -3.12253737e+00,
        -5.98421907e+00, -7.99421892e-02, -3.68708348e+00,  4.74768400e+00,
        -6.53008938e+00, -5.08387709e+00,  4.87733096e-01,  7.51088953e+00,
        -8.85352790e-01,  2.39381790e+00, -6.39236736e+00, -2.13185430e+00,
         2.27601957e+00, -7.91079581e-01, -1.54006314e+00,  1.32115340e+00,
         5.34525514e-01,  1.85582459e+00, -4.75857115e+00, -4.39922333e+00,
        -5.72198153e+00,  3.75011295e-01, -5.80002642e+00,  1.23093772e+00,
        -5.30414248e+00,  2.22271156e+00], dtype=float32),
 array([-4.1700519e+02,  9.93

In [ ]:
x = np.array(ext_df["feature"].tolist())
y = np.array(ext_df["class"].tolist())

In [ ]:
print(f"class before encoding:\n{y[1]}")

le = LabelEncoder()
y = to_categorical(le.fit_transform(y))

print(f"\nclass before encoding:\n{y[1]}")

class before encoding:
children_playing

class before encoding:
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
# splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

print(f"Number of training samples: {X_train.shape[0]}")
print(f"Number of testing samples: {X_test.shape[0]}")

Number of training samples: 6985
Number of testing samples: 1747


In [ ]:
# creating model
num_labels = y.shape[1]
model = Sequential()

model.add(Dense(128, input_shape=(50,)))
model.add(Activation("relu"))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(128))

model.add(Dense(num_labels))
model.add(Activation("softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               6528      
                                                                 
 activation (Activation)     (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 activation_1 (Activation)   (None, 256)               0         
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               6

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"],
)

In [ ]:
num_epochs = 250
num_batch_size = 32

model.fit(
    X_train,
    y_train,
    batch_size=num_batch_size,
    epochs=num_epochs,
    validation_data=(X_test, y_test),
)

Epoch 1/250
219/219 [==============================] - 8s 7ms/step - loss: 7.9322 - accuracy: 0.1449 - val_loss: 2.1426 - val_accuracy: 0.2318
Epoch 2/250
219/219 [==============================] - 1s 6ms/step - loss: 2.3319 - accuracy: 0.2029 - val_loss: 2.1288 - val_accuracy: 0.1815
Epoch 3/250
219/219 [==============================] - 1s 4ms/step - loss: 2.1059 - accuracy: 0.2478 - val_loss: 1.9821 - val_accuracy: 0.2335
Epoch 4/250
219/219 [==============================] - 1s 4ms/step - loss: 1.9689 - accuracy: 0.2913 - val_loss: 1.8114 - val_accuracy: 0.3177
Epoch 5/250
219/219 [==============================] - 1s 4ms/step - loss: 1.8503 - accuracy: 0.3392 - val_loss: 1.6667 - val_accuracy: 0.4098
Epoch 6/250
219/219 [==============================] - 1s 4ms/step - loss: 1.7370 - accuracy: 0.3884 - val_loss: 1.5399 - val_accuracy: 0.4625
Epoch 7/250
219/219 [==============================] - 1s 4ms/step - loss: 1.6700 - accuracy: 0.4235 - val_loss: 1.4788 - val_accuracy: 0.5283

In [ ]:
def extract_feature(file_name):
    audio_data, sample_rate = librosa.load(file_name)
    fea = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=50)
    scaled = np.mean(fea.T, axis=0)
    return np.array([scaled])

In [ ]:
def print_prediction(file_name):
    pred_fea = extract_feature(file_name)
    pred_vector = np.argmax(model.predict(pred_fea), axis=-1)
    pred_class = le.inverse_transform(pred_vector)
    print("The predicted class is:", pred_class[0], "\n")

In [ ]:
file_name = "./urbansound8k/fold1/102842-3-1-6.wav"

print_prediction(file_name)

ipd.Audio(file_name)

1/1 [==============================] - 0s 85ms/step
The predicted class is: dog_bark 



In [ ]:
import random

audios = glob.glob(os.path.join("./urbansound8k/*/*.wav"), recursive=True)
file_name = random.choice(audios)
print_prediction(file_name)
ipd.Audio(file_name)

1/1 [==============================] - 0s 121ms/step
The predicted class is: engine_idling 



In [ ]:
# saving the encoder
import pickle

pickle.dump(le, open("label_encoder.pkl", "wb"))

# saving the model
model.save("model.h5")

In [ ]:
import pickle
import random

model = tf.keras.models.load_model("model.h5")
le = pickle.load(open("label_encoder.pkl", "rb"))

In [ ]:
def extract_feature(file_name):
    audio_data, sample_rate = librosa.load(file_name)
    fea = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=50)
    scaled = np.mean(fea.T, axis=0)
    return np.array([scaled])


def print_prediction(file_name):
    pred_fea = extract_feature(file_name)
    pred_vector = np.argmax(model.predict(pred_fea), axis=-1)
    pred_class = le.inverse_transform(pred_vector)
    print("The predicted class is:", pred_class[0], "\n")


audios = glob.glob(os.path.join("./urbansound8k/*/*.wav"), recursive=True)
file_name = random.choice(audios)
print(f"file name: {file_name}\n")
print_prediction(file_name)
ipd.Audio(file_name)

file name: ./urbansound8k/fold1/165067-2-0-35.wav

1/1 [==============================] - 0s 34ms/step
The predicted class is: children_playing 

